In [3]:
import requests
from urllib.parse import unquote
import unidecode
import re
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [2]:
def get_wikipedia(url="",name="",lang="en"):
    if url:
        subject = unquote(url.split("/")[-1].replace("_"," "))
    elif name:
        subject = name
    else:
        return {"response":[],"names":[]}
    
    base = 'https://'+lang+'.wikipedia.org/w/api.php'
    #searching entire wikipedia
    #print("Searching ",subject)
    params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'srsearch':subject
        }
 
    data = requests.get(base, params=params).json()
    #print(data)
    entry=""
    pageid=""
    if not "query" in data.keys():
        return None
    for reg in data["query"]["search"]:
        score=fuzz.ratio(reg["title"].lower(),subject.lower())
        if score>90:
            entry=reg
            pageid=int(reg["pageid"])
        elif score>50:
            score=fuzz.partial_ratio(reg["title"].lower(),subject.lower())
            if score>95:
                entry=reg
                pageid=int(reg["pageid"])
            elif score>80:
                score=fuzz.token_set_ratio(reg["title"].lower(),subject.lower())
                if score>98:
                    entry=reg
                    pageid=int(reg["pageid"])
        if entry!="":
            break

    if pageid!="":
        #retrieveing the actual page    
        params = {
                'action': 'query',
                'format': 'json',
                'pageids': pageid,
                'prop': 'langlinks',
                'lllimit':500,
                #'exintro': True,
                #'explaintext': True,
            }

        response = requests.get(base, params=params)
        data = response.json()
        return data
    else:
        return None

In [ ]:
wikipedialess=[]
wrong=[]
successful=0

In [4]:
def get_logo_wikipedia(url="",name="",lang="en"):
    if url:
        subject = unquote(url.split("/")[-1].replace("_"," "))
    elif name:
        subject = name
    else:
        return {"response":[],"names":[]}
    
    base = 'https://'+lang+'.wikipedia.org/w/api.php'
    #searching entire wikipedia
    #print("Searching ",subject)
    params = {
            'action':'query',
            'format':'json',
            'list':'search',
            'srsearch':subject
        }
 
    data = requests.get(base, params=params).json()
    #print(data)
    entry=""
    pageid=""
    if not "query" in data.keys():
        return None
    for reg in data["query"]["search"]:
        score=fuzz.ratio(reg["title"].lower(),subject.lower())
        if score>90:
            entry=reg
            pageid=int(reg["pageid"])
        elif score>50:
            score=fuzz.partial_ratio(reg["title"].lower(),subject.lower())
            if score>95:
                entry=reg
                pageid=int(reg["pageid"])
            elif score>80:
                score=fuzz.token_set_ratio(reg["title"].lower(),subject.lower())
                if score>98:
                    entry=reg
                    pageid=int(reg["pageid"])
        if entry!="":
            break

    if pageid!="":
        #retrieveing the actual page    
        params = {
            'action': 'query',
            'format': 'json',
            'pageids': pageid,
            'prop': 'images'
        }

        response = requests.get(base, params=params)
        data = response.json()
        try:
            title=""
            for img in data["query"]["pages"][str(pageid)]["images"]:
                if "commons" in img["title"].lower():
                    continue
                if "logo" in img["title"].lower():
                    title=img["title"]
                    break
                if "escudo" in img["title"].lower():
                    title=img["title"]
            print(title)
            params = {
                'action': 'query',
                'format': 'json',
                'titles': title,
                'prop': 'imageinfo',
                'iiprop':"url"
            }
            response = requests.get(base, params=params)
            data = response.json()
            return data
        except Exception as e:
            print("@@@@")
            print("Function error: ", e)
            print(data)
            print("@@@@")
    else:
        return None